In [5]:
import pandas as pd
import numpy as np
def wilder_rma(series, window):
    return series.ewm(alpha=1/window, adjust=False).mean()

def calc_rsi(data, window=14):
    delta = data['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = wilder_rma(pd.Series(gain), window)
    avg_loss = wilder_rma(pd.Series(loss), window)
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calc_macd(data, fast=12, slow=26, signal=9):
    ema_fast = data['close'].ewm(span=fast, adjust=False).mean()
    ema_slow = data['close'].ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def backtest_combined_cumulative(data, rsi_series, macd_series, signal_series,
                                  rsi_buy, rsi_sell, initial_capital=10000):
    capital = initial_capital
    position = False
    buy_price = 0

    for i in range(1, len(data)):
        close_price = data['close'].iloc[i]

        # Alım koşulu
        if not position and rsi_series[i] < rsi_buy and macd_series[i] > signal_series[i]:
            position = True
            buy_price = close_price

        # Satış koşulu
        elif position and rsi_series[i] > rsi_sell and macd_series[i] < signal_series[i]:
            position = False
            sell_price = close_price
            pct_change = (sell_price - buy_price) / buy_price
            capital *= (1 + pct_change)  # bileşik büyüme

    return capital - initial_capital  # net kâr ($ olarak)


best_result = {'return': -np.inf}
results = []
data = pd.read_csv('./data_set/Nvidia.csv')
data = data.head(1000).reset_index(drop=True)
for rsi_window in range(5, 31, 5):
    rsi = calc_rsi(data, rsi_window)

    for rsi_buy in range(10, 51, 10):
        for rsi_sell in range(60, 91, 10):
            if rsi_buy >= rsi_sell:
                continue

            for macd_fast in range(6, 19, 2):
                for macd_slow in range(macd_fast + 2, 41, 2):
                    for macd_signal in range(4, 21, 4):

                        macd, signal = calc_macd(data, macd_fast, macd_slow, macd_signal)
                        ret = backtest_combined_cumulative(data, rsi, macd, signal, rsi_buy, rsi_sell)

                        results.append({
                            'rsi_window': rsi_window,
                            'rsi_buy': rsi_buy,
                            'rsi_sell': rsi_sell,
                            'macd_fast': macd_fast,
                            'macd_slow': macd_slow,
                            'macd_signal': macd_signal,
                            'return': ret
                        })

                        if ret > best_result['return']:
                            best_result = {
                                'rsi_window': rsi_window,
                                'rsi_buy': rsi_buy,
                                'rsi_sell': rsi_sell,
                                'macd_fast': macd_fast,
                                'macd_slow': macd_slow,
                                'macd_signal': macd_signal,
                                'return': ret
                            }

df = pd.DataFrame(results)
print(f"📈 En iyi kombinasyon:\n"
      f"RSI window = {best_result['rsi_window']}, RSI alım < {best_result['rsi_buy']}, RSI satış > {best_result['rsi_sell']}\n"
      f"MACD fast = {best_result['macd_fast']}, slow = {best_result['macd_slow']}, signal = {best_result['macd_signal']}\n"
      f"Toplam Getiri: {best_result['return']:.2f}")


📈 En iyi kombinasyon:
RSI window = 5, RSI alım < 50, RSI satış > 80
MACD fast = 6, slow = 12, signal = 12
Toplam Getiri: 102464.55
